In [1]:
import os
import sys
import time
import numpy as np
import scipy.sparse as sp
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

Using TensorFlow backend.


In [2]:
# the path of execution
EXE_PATH = os.path.abspath(os.path.curdir)
# the path of the vendor files
VENDOR_PATH = os.path.join(EXE_PATH, 'vendor')
# the vendors to include in the system path
VENDORS = ['keras-gcn']
# create the absolute paths for all vendors
VENDORS = list(map(lambda x: os.path.join(VENDOR_PATH, x), VENDORS))
# update the Python path to include necessary vendor module
sys.path += VENDORS

In [3]:
from kegra.layers.graph import GraphConvolution
from kegra.utils import load_data, preprocess_adj, get_splits, evaluate_preds
from ggcn import GaussianGraphConvolution, kl_reg

# Data

In [4]:
X, A, y = load_data('data/cora/', dataset='cora')
X /= X.sum(1).reshape(-1, 1)
A = preprocess_adj(A)
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)

Loading cora dataset...
Dataset has 2708 nodes, 5429 edges, 1433 features.


In [5]:
X

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
A

<2708x2708 sparse matrix of type '<class 'numpy.float64'>'
	with 13264 stored elements in Compressed Sparse Row format>

In [7]:
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [8]:
support = 1
graph = [X, A]
G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True)]

W1110 18:17:57.215714 139680131151680 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1110 18:17:57.222760 139680131151680 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:515: The name tf.sparse_placeholder is deprecated. Please use tf.compat.v1.sparse_placeholder instead.



In [9]:
X_in = Input(shape=(X.shape[1],))

W1110 18:17:57.227551 139680131151680 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



# GCN (baseline)

In [10]:
# Define model architecture
# NOTE: We pass arguments for graph convolutional layers as a list of tensors.
# This is somewhat hacky, more elegant options would require rewriting the Layer base class.
H = Dropout(0.5)(X_in)
H = GraphConvolution(32, support, activation='relu', kernel_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GraphConvolution(y.shape[1], support, activation='softmax')([H]+G)

W1110 18:17:57.232685 139680131151680 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1110 18:17:57.236961 139680131151680 deprecation.py:506] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1110 18:17:57.245815 139680131151680 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [11]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

W1110 18:17:57.286554 139680131151680 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [12]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1433)         0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
graph_convolution_1 (GraphConvo (None, 32)           45888       dropout_1[0][0]                  
                                                                 input_1[0][0]                    
__________

In [ ]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

W1110 18:17:57.359720 139680131151680 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch: 0001 train_loss= 1.9277 train_acc= 0.3071 val_loss= 1.9293 val_acc= 0.3500 time= 1.0580
Epoch: 0002 train_loss= 1.9079 train_acc= 0.3000 val_loss= 1.9119 val_acc= 0.3500 time= 0.0170
Epoch: 0003 train_loss= 1.8853 train_acc= 0.2929 val_loss= 1.8922 val_acc= 0.3500 time= 0.0164
Epoch: 0004 train_loss= 1.8617 train_acc= 0.2929 val_loss= 1.8709 val_acc= 0.3500 time= 0.0164
Epoch: 0005 train_loss= 1.8386 train_acc= 0.2929 val_loss= 1.8499 val_acc= 0.3500 time= 0.0166
Epoch: 0006 train_loss= 1.8157 train_acc= 0.2929 val_loss= 1.8292 val_acc= 0.3500 time= 0.0166
Epoch: 0007 train_loss= 1.7932 train_acc= 0.2929 val_loss= 1.8096 val_acc= 0.3500 time= 0.0170
Epoch: 0008 train_loss= 1.7720 train_acc= 0.2929 val_loss= 1.7916 val_acc= 0.3500 time= 0.0172
Epoch: 0009 train_loss= 1.7522 train_acc= 0.2929 val_loss= 1.7754 val_acc= 0.3500 time= 0.0170
Epoch: 0010 train_loss= 1.7343 train_acc= 0.2929 val_loss= 1.7616 val_acc= 0.3500 time= 0.0174
Epoch: 0011 train_loss= 1.7183 train_acc= 0.2929 v

Epoch: 0098 train_loss= 0.6350 train_acc= 0.9286 val_loss= 0.9445 val_acc= 0.7933 time= 0.0180
Epoch: 0099 train_loss= 0.6315 train_acc= 0.9214 val_loss= 0.9394 val_acc= 0.7967 time= 0.0163
Epoch: 0100 train_loss= 0.6271 train_acc= 0.9214 val_loss= 0.9355 val_acc= 0.7967 time= 0.0170
Epoch: 0101 train_loss= 0.6220 train_acc= 0.9214 val_loss= 0.9317 val_acc= 0.7967 time= 0.0167
Epoch: 0102 train_loss= 0.6149 train_acc= 0.9286 val_loss= 0.9280 val_acc= 0.7967 time= 0.0169
Epoch: 0103 train_loss= 0.6085 train_acc= 0.9286 val_loss= 0.9255 val_acc= 0.7933 time= 0.0166
Epoch: 0104 train_loss= 0.6034 train_acc= 0.9214 val_loss= 0.9241 val_acc= 0.7933 time= 0.0172
Epoch: 0105 train_loss= 0.5990 train_acc= 0.9286 val_loss= 0.9219 val_acc= 0.7933 time= 0.0174
Epoch: 0106 train_loss= 0.5942 train_acc= 0.9357 val_loss= 0.9191 val_acc= 0.7933 time= 0.0165
Epoch: 0107 train_loss= 0.5890 train_acc= 0.9286 val_loss= 0.9155 val_acc= 0.7967 time= 0.0172
Epoch: 0108 train_loss= 0.5841 train_acc= 0.9357 v

In [ ]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

# Gaussian Graph Convolution

In [ ]:
B1 = 5e-4
B2 = 5e-4

In [ ]:
H = Dropout(0.5)(X_in)
H1 = GaussianGraphConvolution(16,
    is_first=True,
    activation='relu',
    mean_regularizer=l2(B1),
    variance_regularizer=l2(B1),
#     dropout=0.5
)([H]+G)
Y = GaussianGraphConvolution(y.shape[1],
    is_last=True,
    activation='softmax',
)(H1+G)

In [ ]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))
model.add_loss(kl_reg(*H1, B2), H1)

In [ ]:
model.summary()

In [ ]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

In [ ]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

# Attacks